In [ ]:
import pandas as pd
import numpy as np
import re
from textblob import TextBlob
import io
import random


# Sample economic news data
data_str = """Date Time Currency Impact Event Title Actual Forecast Previous
Apr-22 Day 2 All Medium IMF Meetings
Apr-20 1:00am USD Medium FOMC Member Fischer Speaks
Apr-20 3:45am NZD High CPI q/q 1.00% 0.80% 0.40%
Apr-20 4:50am JPY Low Trade Balance 0.17T 0.61T 0.61T
Apr-20 6:30am AUD Medium NAB Quarterly Business Confidence 6 6
Apr-20 11:00am EUR Low German PPI m/m 0.00% 0.20% 0.20%
Apr-20 1:50pm EUR Low Spanish 10-y Bond Auction 1.68|1.5 1.61|1.6
Apr-20 5:30pm USD High Philly Fed Manufacturing Index 22 25.6 32.8
Apr-20 7:00pm EUR Low Consumer Confidence -4 -5 -5
Apr-20 7:30pm USD Low Natural Gas Storage 54B 49B 10B
Apr-20 8:30pm GBP High BOE Gov Carney Speaks
Apr-20 9:30pm GBP High BOE Gov Carney Speaks
Apr-20 10:15pm USD High Treasury Sec Mnuchin Speaks
Apr-26 1:30am USD Low API Weekly Statistical Bulletin
Apr-26 3:45am NZD Low Visitor Arrivals m/m 1.50%   -1.90%
Apr-26 6:30am AUD High CPI q/q 0.50% 0.60% 0.50%"""


df = pd.read_csv('forexfactory_calendar_full.csv')

# Function to normalize values and extract numeric part
def normalize_value(value):
    if pd.isna(value):
        return None

    # Convert to string and remove any non-numeric characters except decimal point and negative sign
    if isinstance(value, str):
        # Extract percentage if present
        if '%' in value:
            try:
                return float(value.replace('%', '')) / 100
            except:
                return None

        # Extract numeric part for values with T, B, etc.
        match = re.search(r'(-?\d+\.?\d*)', value)
        if match:
            return float(match.group(1))

    try:
        return float(value)
    except:
        return None

# Function to calculate sentiment score based on economic indicators
def calculate_sentiment_score(row):
    # Base sentiment score
    score = 0

    # Factor 1: Impact weighting
    impact_weight = {'High': 0.3, 'Medium': 0.2, 'Low': 0.1}
    if row['Impact'] in impact_weight:
        score += impact_weight[row['Impact']]

    # Factor 2: Currency importance (USD and EUR are considered more impactful)
    currency_weight = {'USD': 0.15, 'EUR': 0.12, 'GBP': 0.1, 'JPY': 0.08, 'AUD': 0.07, 'NZD': 0.06}
    if row['Currency'] in currency_weight:
        score += currency_weight[row['Currency']]

    # Factor 3: Actual vs Forecast comparison
    actual = normalize_value(row['Actual'])
    forecast = normalize_value(row['Forecast'])

    if actual is not None and forecast is not None:
        # Determine if higher or lower values are better based on the event type
        event_title = row['Event Title'].lower()

        # Generally positive indicators when higher than forecast
        positive_indicators = ['gdp', 'consumer confidence', 'business confidence', 'manufacturing', 'employment', 'retail sales']
        # Generally negative indicators when higher than forecast
        negative_indicators = ['unemployment', 'deficit', 'inflation', 'cpi', 'ppi']

        is_positive_indicator = any(term in event_title for term in positive_indicators)
        is_negative_indicator = any(term in event_title for term in negative_indicators)

        if is_positive_indicator:
            # For positive indicators, higher than forecast is good
            if actual > forecast:
                score += 0.2 * (actual - forecast) / max(abs(forecast), 0.001)
            else:
                score -= 0.2 * (forecast - actual) / max(abs(forecast), 0.001)
        elif is_negative_indicator:
            # For negative indicators, lower than forecast is good
            if actual < forecast:
                score += 0.2 * (forecast - actual) / max(abs(forecast), 0.001)
            else:
                score -= 0.2 * (actual - forecast) / max(abs(forecast), 0.001)
        else:
            # For other indicators, any deviation from forecast is treated as slightly negative
            score -= 0.1 * abs(actual - forecast) / max(abs(forecast), 0.001)

    # Factor 4: Sentiment from event title text
    title_sentiment = TextBlob(row['Event Title']).sentiment.polarity
    score += 0.25 * title_sentiment

    # Normalize score to range between -1 and 1
    score = max(min(score, 1), -1)

    # Apply randomization to extreme values
    if abs(score) == 1.0:
        if score > 0:
            score *= random.uniform(0.85, 0.99)  # Randomize extreme positive values
        else:
            score *= random.uniform(0.85, 0.99)  # Randomize extreme negative values

    return score

# Function to create a summary from the event title
def create_summary(row):
    event = row['Event Title']
    currency = row['Currency']
    actual = row['Actual']
    forecast = row['Forecast']

    # For events with actual and forecast values
    if pd.notna(actual) and pd.notna(forecast):
        actual_val = normalize_value(actual)
        forecast_val = normalize_value(forecast)

        if actual_val is not None and forecast_val is not None:
            if actual_val > forecast_val:
                performance = "better than expected"
            elif actual_val < forecast_val:
                performance = "worse than expected"
            else:
                performance = "as expected"

            return f"{currency} {event} reported {performance} at {actual}."

    # For events without actual/forecast values
    return f"{currency} {event}."

# Apply sentiment analysis to each row
df['Sentiment Score'] = df.apply(calculate_sentiment_score, axis=1)
df['News Headline Summary'] = df.apply(create_summary, axis=1)

# Select and format the output
result_df = df[['Sentiment Score', 'News Headline Summary']].copy()
result_df['Sentiment Score'] = result_df['Sentiment Score'].round(2)

# Sort by sentiment score
result_df = result_df.sort_values(by='Sentiment Score', ascending=False)

# Function to create additional entries with slightly varied sentiment scores
def expand_dataset(df, target_size=68000):
    original_size = len(df)
    copies_needed = target_size - original_size

    if copies_needed <= 0:
        return df

    duplication_factor = int(np.ceil(copies_needed / original_size))

    # Initialize an empty list to hold all rows
    expanded_data = [df]

    # Generate duplicates with variations
    for _ in range(duplication_factor):
        # Create a copy of the original dataframe
        df_copy = df.copy()

        # Add random variations to sentiment scores
        df_copy['Sentiment Score'] = df_copy['Sentiment Score'].apply(
            lambda x: max(min(x + random.uniform(-0.15, 0.15), 1.0), -1.0)
        )

        # Round to 2 decimal places
        df_copy['Sentiment Score'] = df_copy['Sentiment Score'].round(2)

        expanded_data.append(df_copy)

    # Combine all dataframes
    expanded_df = pd.concat(expanded_data, ignore_index=True)

    # Trim to exact target size
    return expanded_df.sample(target_size)

# Expand the dataset to target size
expanded_result_df = expand_dataset(result_df, target_size=68000)

# Sort the final expanded dataset
final_df = expanded_result_df.sort_values(by='Sentiment Score', ascending=False)

print(f"Original dataset size: {len(result_df)}")
print(f"Expanded dataset size: {len(final_df)}")

final_df.to_csv('sentiment_analysis_results.csv', index=False)

print("Results saved to 'sentiment_analysis_results.csv'")
print(final_df.head(10))  # Print first 10 rows

Original dataset size: 26844
Expanded dataset size: 68000
Results saved to 'sentiment_analysis_results.csv'
       Sentiment Score                              News Headline Summary
53706              1.0  CHF Retail Sales y/y reported better than expe...
26878              1.0  EUR Italian Retail Sales m/m reported better t...
26869              1.0  GBP Retail Sales m/m reported better than expe...
26864              1.0  USD Empire State Manufacturing Index reported ...
53730              1.0  EUR Retail Sales m/m reported better than expe...
53787              1.0  CAD Employment Change reported better than exp...
26844              1.0  USD PPI m/m reported worse than expected at -0...
26879              1.0  EUR German Unemployment Change reported better...
26899              1.0  GBP Retail Sales m/m reported better than expe...
26868              1.0  CHF Retail Sales y/y reported better than expe...


In [ ]:
import pandas as pd

# Load CSV
df = pd.read_csv("btc_historical_data.csv")

# Split 'Date Time' into 'Date' and 'Time'
df[['Date', 'Time']] = df['Date Time'].str.split(' ', expand=True)

# Drop the original 'Date Time' column
df = df.drop(columns=['Date Time'])

# Reorder columns if needed
df = df[['Date', 'Time', 'Crypto', 'Open Price', 'Close Price', 'High Price', 'Low Price', 'Volume']]

# Show result
print(df)

# Optionally save
df.to_csv("output.csv", index=False)


           Date   Time   Crypto  Open Price  Close Price  High Price  \
0      17-08-17  04:00  Bitcoin        4261         4309        4314   
1      17-08-17  05:00  Bitcoin        4309         4315        4329   
2      17-08-17  06:00  Bitcoin        4330         4324        4345   
3      17-08-17  07:00  Bitcoin        4317         4350        4350   
4      17-08-17  08:00  Bitcoin        4333         4361        4378   
...         ...    ...      ...         ...          ...         ...   
67460  03-05-25  08:00  Bitcoin       96323        96281       96422   
67461  03-05-25  09:00  Bitcoin       96281        96254       96396   
67462  03-05-25  10:00  Bitcoin       96254        95896       96254   
67463  03-05-25  11:00  Bitcoin       95896        95942       96084   
67464  03-05-25  12:00  Bitcoin       95942        96155       96164   

       Low Price       Volume  
0           4261    47.18 BTC  
1           4291    23.23 BTC  
2           4309     7.23 BTC  
3      

In [ ]:
import pandas as pd

# Load the first file (crypto data)
df_prices = pd.read_csv("output.csv")

# Load the second file (sentiment data)
df_sentiment = pd.read_csv("sentiment_analysis_results.csv")

# Make sure both have same number of rows, or truncate/pad if needed
min_len = min(len(df_prices), len(df_sentiment))
df_prices = df_prices.iloc[:min_len].reset_index(drop=True)
df_sentiment = df_sentiment.iloc[:min_len].reset_index(drop=True)

# Concatenate side by side
df_merged = pd.concat([df_prices, df_sentiment], axis=1)

# Save to new CSV
df_merged.to_csv("merged_output.csv", index=False)

print(df_merged)


           Date   Time   Crypto  Open Price  Close Price  High Price  \
0      17-08-17  04:00  Bitcoin        4261         4309        4314   
1      17-08-17  05:00  Bitcoin        4309         4315        4329   
2      17-08-17  06:00  Bitcoin        4330         4324        4345   
3      17-08-17  07:00  Bitcoin        4317         4350        4350   
4      17-08-17  08:00  Bitcoin        4333         4361        4378   
...         ...    ...      ...         ...          ...         ...   
67460  03-05-25  08:00  Bitcoin       96323        96281       96422   
67461  03-05-25  09:00  Bitcoin       96281        96254       96396   
67462  03-05-25  10:00  Bitcoin       96254        95896       96254   
67463  03-05-25  11:00  Bitcoin       95896        95942       96084   
67464  03-05-25  12:00  Bitcoin       95942        96155       96164   

       Low Price       Volume  Sentiment Score  \
0           4261    47.18 BTC             1.00   
1           4291    23.23 BTC      

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load dataset
data = pd.read_csv("merged_output.csv")

# Convert Date and Time to datetime format
data['Datetime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'], format='%d-%m-%y %H:%M')
data.drop(['Date', 'Time'], axis=1, inplace=True)

# Encode categorical features
le = LabelEncoder()
data['Crypto'] = le.fit_transform(data['Crypto'])
data['News Headline Summary'] = le.fit_transform(data['News Headline Summary'])

# Convert 'Volume' to numeric by removing non-numeric characters
data['Volume'] = data['Volume'].str.replace(r'\D', '', regex=True).astype(float)

# Define features and target
X = data.drop(['Close Price', 'Datetime'], axis=1)  # Exclude Datetime
y = data['Close Price']

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train SVM (Regression)
svm_model = SVR()
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)

# Train ANN (Regression)
ann_model = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500)
ann_model.fit(X_train, y_train)
ann_pred = ann_model.predict(X_test)

# Train Bayesian Regression model
bayesian_model = BayesianRidge()
bayesian_model.fit(X_train, y_train)
bayesian_pred = bayesian_model.predict(X_test)

# Evaluation function for regression models
def evaluate_model(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    print(f"{model_name} Results:")
    print(f"MAE (USD): {mae:.4f}")
    print(f"RMSE (USD): {rmse:.4f}")
    print(f"MAPE (%): {mape:.4f}%\n")

# Evaluate models
evaluate_model(y_test, svm_pred, "SVM (Regression)")
evaluate_model(y_test, ann_pred, "ANN (Regression)")
evaluate_model(y_test, bayesian_pred, "Bayesian Regression")


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


SVM (Regression) Results:
MAE (USD): 12603.7330
RMSE (USD): 20362.7537
MAPE (%): 56.9663%

ANN (Regression) Results:
MAE (USD): 66.2358
RMSE (USD): 120.8542
MAPE (%): 0.2460%

Bayesian Regression Results:
MAE (USD): 60.8688
RMSE (USD): 112.7585
MAPE (%): 0.2178%

